In [ ]:
import pandas as pd
import plotly.express as px
import requests
from openai import OpenAI


In [ ]:
def get_column_name(promt:str = ""):
    client = OpenAI(
        api_key=""
        base_url=""
    )

    response = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[
            {"role": "user", "content": promt}
        ]
    )

    return response.choices[0].message.content.strip()

In [ ]:
#Import
df_transactions_raw = pd.DataFrame(pd.read_excel("data/Bestandstransaktionshistorie 2025.xlsx"))
df_material_master_raw = pd.DataFrame(pd.read_excel("data/SCM_Material_Master.xlsx"))
df_prices_raw =pd.DataFrame(pd.read_excel("data/SCM_Prices.xlsx"))

In [ ]:
df_data_out_cleaned =df_transactions_raw.head(10).dropna(axis=1, how='all')
df_material_master_cleaned = df_material_master_raw.head(10).dropna(axis=1, how='all')
df_prices_cleaned = df_prices_raw.head(10).dropna(axis=1, how='all')


In [ ]:
df_sample_out = df_data_out_cleaned.head(10)
csv_text_out = df_sample_out.to_csv(index=False)

df_master_sample = df_material_master_cleaned.head(10)
csv_text_master = df_master_sample.to_csv(index=False)

df_prices_sample = df_prices_cleaned.head(10)
csv_text_prices = df_prices_sample.to_csv(index=False)

In [ ]:
###Prompts

#Prompts Data Out
out_date_prompt = f"""
We have this Data in csv format {csv_text_out} which is a transactions list for outgoing material.
You have the Column Titles aswell as the first 10 rows.
please return the most likely column name containing the date when the material is used based on titles and the data.
return only the column name that you think is the most likely.
A column can only be relevant if it contains Dates. A Date should at least consist of year, month and day.
Your Answer should contain only the column name EXACTLY as it is in the Dataset.
"""

out_article_prompt = f'''
We have this Data in csv format {csv_text_out} which is a transactions list for outgoing material.
can you provide me the name of the column most likely conataining the Material Number / Article Number ?
You anser should only be the name of the column.
'''
out_quantity_prompt = f'''
We have this Data in csv format {csv_text_out} which is a transactions list for outgoing material.
can you provide me the name of the column most likely conataining the Material Quantity ?
You anser should only be the name of the column.
'''

out_unit_promt = f'''
We have this Data in csv format {csv_text_out} which is a transactions list for outgoing material.
can you provide me the name of the column most likely conataining the unit the material quantity is measured in?
You anser should only be the name of the column.
'''



#Prompts Master Data

master_article_prompt = f'''
We have this Data in csv format {csv_text_master} which is material master data.
can you provide me the name of the column most likely conataining the Material Number / Article Number ?
Your anser should only be the name of that column.
'''

master_price_prompt = f'''
We have this Data in csv format {csv_text_master} which is material master data.
can you provide me the name of the column most likely conataining the Price?
Your anser should only be the name of that column.
'''

#Prompt Prices

prices_article_prompt = f'''
We have this Data in csv format {csv_text_prices} which is pricing data.
can you provide me the name of the column most likely conataining the Material Number / Article Number ?
Your anser should only be the name of that column.
'''

prices_price_prompt = f'''
We have this Data in csv format {csv_text_prices} which is pricing data.
can you provide me the name of the column most likely conataining the Price?
Your anser should only be the name of that column.
'''


In [ ]:
#Get column Names

#Out
article_out_col = get_column_name(out_article_prompt)
quantity_out_col = get_column_name(out_quantity_prompt)
date_out_col = get_column_name(out_date_prompt)
unit_out_col = get_column_name(out_unit_promt)

print("Obtained Column Names from Transactions: ", article_out_col, quantity_out_col, date_out_col, unit_out_col)

#Prices
article_prices_col = get_column_name(prices_article_prompt)
price_prices_col = get_column_name(prices_price_prompt)
print("Obtained Column Names from Prices: ", article_prices_col, price_prices_col)

In [ ]:
#Create Main Dataset
df_traffic_out = df_transactions_raw[[article_out_col, date_out_col, quantity_out_col, unit_out_col]].copy()
df_traffic_out[date_out_col] = pd.to_datetime(df_traffic_out[date_out_col]).dt.date
df_traffic_out.rename(columns={article_out_col:"Article", quantity_out_col:"Quantity", date_out_col:"Date", unit_out_col:"Unit"}, inplace = True)

df_prices = df_prices_raw[[article_prices_col, price_prices_col]].copy()
df_prices.rename(columns = {article_prices_col:"Article", price_prices_col:"Price"}, inplace = True)

df_traffic_out = pd.merge(df_traffic_out,df_prices, how = 'left', on = 'Article')
df_traffic_out['Volume'] = df_traffic_out['Quantity'] * df_traffic_out['Price']

In [ ]:
df_traffic_out

In [ ]:
###XYZ###
def get_classification_abc_xyz_materials(traffic_out:pd.DataFrame, lower_bound_xyz: int = 3, upper_bound_xyz:int = 9, lower_bound_abc:float= 0.05, upper_bound_abc:float  = 0.15 ) -> pd.DataFrame:


    ##XYZ
    df_consumption_xyz = traffic_out[['Article',  'Quantity']].copy()
    df_consumption_xyz.loc[:,'Count Verbrauch'] = (df_consumption_xyz['Quantity'] > 0).astype(int)
    df_consumption_xyz =df_consumption_xyz[['Article', 'Count Verbrauch']].groupby('Article').sum().reset_index()
    df_consumption_xyz['XYZ'] = 'Z'
    df_consumption_xyz.loc[df_consumption_xyz['Count Verbrauch'] > lower_bound_xyz, 'XYZ'] = 'Y'
    df_consumption_xyz.loc[df_consumption_xyz['Count Verbrauch'] > upper_bound_xyz, 'XYZ'] = 'X'
    df_consumption_xyz = df_consumption_xyz[['Article', 'XYZ']]
    

    #ABC
    df_consumption_abc = traffic_out[['Article',  'Volume']].copy()
    df_consumption_abc = df_consumption_abc.groupby('Article').sum().reset_index()
    df_consumption_abc = df_consumption_abc.sort_values('Volume', ascending= True)
    total_value = df_consumption_abc['Volume'].sum()
    df_consumption_abc['Cumulated'] = df_consumption_abc['Volume'].cumsum()
    df_consumption_abc['ABC'] = 'C'
    df_consumption_abc.loc[df_consumption_abc['Cumulated'] > lower_bound_abc* total_value, 'ABC'] = 'B'
    df_consumption_abc.loc[df_consumption_abc['Cumulated'] > upper_bound_abc* total_value, 'ABC'] = 'A'
    df_consumption_abc.loc[df_consumption_abc['ABC'] =='A']
    df_consumption_abc = df_consumption_abc[['Article', 'ABC']]


    df_abc_xyz_classigication = pd.merge(df_consumption_xyz, df_consumption_abc, how = 'left', on = 'Article')

    return df_abc_xyz_classigication



In [ ]:
#Get ABC XYZ Classification
df_abc_xyz = get_classification_abc_xyz_materials(df_traffic_out)

In [ ]:
#PLOT ABC XYZ Classificaton
import plotly.graph_objects as go
# Count categories in XYZ
xyz_counts =df_abc_xyz['XYZ'].value_counts().reset_index()
xyz_counts.columns = ['Category', 'Count']

# Count categories in ABC
abc_counts = df_abc_xyz['ABC'].value_counts().reset_index()
abc_counts.columns = ['Category', 'Count']

# Create subplots layout
fig = go.Figure()

# Pie chart for XYZ
fig.add_trace(go.Pie(
    labels=xyz_counts['Category'],
    values=xyz_counts['Count'],
    name='XYZ Kategorien',
    domain=dict(x=[0, 0.48])
))

# Pie chart for ABC
fig.add_trace(go.Pie(
    labels=abc_counts['Category'],
    values=abc_counts['Count'],
    name='ABC Kategorien',
    domain=dict(x=[0.52, 1.0])
))

# Layout
fig.update_layout(
    title_text='Verteilung der Artikel: XYZ vs. ABC Kategorien',
    annotations=[
        dict(text='XYZ', x=0.22, y=0.5, font_size=14, showarrow=False),
        dict(text='ABC', x=0.78, y=0.5, font_size=14, showarrow=False)
    ]
)

fig.show()

In [ ]:
#run model locally
response = requests.post(
    'http://localhost:11434/api/generate',
    json={
        'model': 'llama3:8b',         # or 'mistral', 'codellama', etc.
        'prompt': prompt,
        'stream': False          # If True, it streams responses chunk by chunk
    }
)

print(response.json()['response'])

In [ ]:
#Basic Plot

df_transactions = pd.DataFrame()
df_transactions = df_traffic_out.copy()

# Assuming df_transactions is your DataFrame
df_transactions = df_transactions.sort_values('Date')

# Optional: filter by article if needed
df_transactions = df_transactions[df_transactions['Article'] == '0000991168']

# Group by date and sum quantities
daily = df_transactions.groupby('Date', as_index=False)['Quantity'].sum()

# Plot
fig = px.line(
    daily,
    x='Date',
    y='Quantity',
    title='Verbrauch nach Zeit',
    line_shape='hv'  # <--- this gives smooth curves!
)
fig.update_traces(
    mode='lines+markers',
    line=dict(dash='solid', width=6)  # Other options: 'dot', 'dashdot', 'solid'
)
fig.update_layout(
    xaxis_title='Datum',
    yaxis_title='Verbrauch',
    yaxis=dict(range=[0, daily['Quantity'].max() + 1])  # add buffer if needed
)
fig.show()

In [ ]:
#Plot With Filter


import pandas as pd
import plotly.graph_objects as go

df_transactions = pd.DataFrame()
df_transactions = df_traffic_out.copy()

# Group and prepare data
df_transactions = df_transactions.sort_values('Date')
grouped = df_transactions.groupby(['Date', 'Article'], as_index=False)['Quantity'].sum()

# Get unique articles
articles = grouped['Article'].unique()

# Create figure with all article traces (initially visible=False)
fig = go.Figure()

for i, article in enumerate(articles):
    article_data = grouped[grouped['Article'] == article]
    
    fig.add_trace(go.Scatter(
        x=article_data['Date'],
        y=article_data['Quantity'],
        mode='lines+markers',
        name=str(article),
        line=dict(width=6),
        visible=(i == 0)  # Show only the first article initially
    ))

# Add dropdown menu to select article
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "method": "update",
                    "label": str(article),
                    "args": [
                        {"visible": [i == j for j in range(len(articles))]},
                        {"title": f"Verbrauch nach Zeit – Artikel {article}"}
                    ],
                }
                for i, article in enumerate(articles)
            ],
            "direction": "down",
            "showactive": True,
            "x": 0.5,
            "xanchor": "left",
            "y": 1.2,
            "yanchor": "top"
        }
    ],
    title="Verbrauch nach Zeit – Artikel " + str(articles[0]),
    xaxis_title="Datum",
    yaxis_title="Verbrauch"
)

fig.show()
